In [2]:
pip install transformers requests beautifulsoup4

In [56]:
import torch
import numpy as np
import pandas as pd


In [34]:
from transformers import AutoTokenizer,AutoModelForTokenClassification,AutoModelForSequenceClassification
import re
import requests
from bs4 import BeautifulSoup 

In [35]:
tokenizer=AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model=AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [40]:
tokens=tokenizer.encode('fuck you',return_tensors='pt')

In [41]:
tokens[0]

tensor([  101, 69338, 10855,   102])

In [42]:
resul=model(tokens)

In [43]:
int(torch.argmax(resul.logits))

0

In [44]:
model(tokens)

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.5705,  0.3913, -0.7715, -1.3880, -0.4779]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [32]:
print(torch.__version__)

1.11.0


In [52]:
r=requests.get('https://www.yelp.com/biz/social-brew-cafe-pyrmont')
soup=BeautifulSoup(r.text,'html.parser')
regex=re.compile('.*comment.*')
result=soup.find_all('p',{'class':regex})
reviews=[results.text for results in result]

In [54]:
reviews[0]

"Great coffee and vibe. That's all \xa0you need. Crab was outstanding but not good finger food like a taco should be. Really want to try the pork belly sandwich - looked excellent. This became my go to breakfast place in Darling harbor. Had the avocado salmon salad breakfast and it was excellent. Service has been excellent."

In [58]:
df=pd.DataFrame(np.array(reviews),columns=['review'])

In [59]:
df

,review
0,Great coffee and vibe. That's all you need. C...
1,Great coffee and vibe. That's all you need. C...
2,I came to Social brew cafe for brunch while ex...
3,Ricotta hot cakes! These were so yummy. I ate ...
4,I went here a little while ago- a beautiful mo...
5,We came for brunch twice in our week-long visi...
6,Ron & Jo are on the go down under and Wow! We...
7,Good coffee and toasts. Straight up and down -...
8,This place is a gem. The ambiance is to die fo...
9,Delicious. The waitress was hot. The burger wa...


In [68]:
def sentiment_score(review):
    tokens=tokenizer.encode(review,return_tensors='pt')
    result=model(tokens)
    return int(torch.argmax(result.logits))

In [69]:
sentiment_score(df['review'].iloc[8])

3

In [70]:
df['sentiment']=df['review'].apply(lambda x: sentiment_score(x[:512]))

In [71]:
df

,review,sentiment
0,Great coffee and vibe. That's all you need. C...,4
1,Great coffee and vibe. That's all you need. C...,3
2,I came to Social brew cafe for brunch while ex...,4
3,Ricotta hot cakes! These were so yummy. I ate ...,4
4,I went here a little while ago- a beautiful mo...,1
5,We came for brunch twice in our week-long visi...,3
6,Ron & Jo are on the go down under and Wow! We...,4
7,Good coffee and toasts. Straight up and down -...,4
8,This place is a gem. The ambiance is to die fo...,2
9,Delicious. The waitress was hot. The burger wa...,3
